In [ ]:

# 🔑 인증 설정
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)


1. 구글스프레드시트 내드라이브/시설/유지/4월마감 조건추출 선택컬럼_2503.csv
   계약번호(D열), 설치주소(M열)

2. 관리고객 링크 : https://docs.google.com/spreadsheets/d/1QZ2sZf2aFVRcdeeXxr8vDlRnpuN35_ZSPWzM3JVwAOw/edit?usp=sharing
   관리고객활동대상_5월,6월 시트, 계약번호(U열) 컬럼과 4월마감 조건추출 선택컬럼_2503.csv  계약번호(D열) 매핑하여 설치주소(M열)을  https://docs.google.com/spreadsheets/d/1QZ2sZf2aFVRcdeeXxr8vDlRnpuN35_ZSPWzM3JVwAOw/edit?usp=sharing
   관리고객활동대상_5월,6월 시트 설치주소 컬럼에 적용




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ✅ 1. 필수 라이브러리 설치 (최초 1회만)
!pip install --upgrade gspread gspread_dataframe oauth2client --quiet

# ✅ 2. 라이브러리 임포트
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from collections import Counter

# ✅ 3. 인증 설정
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)

# ✅ 4. CSV 파일 불러오기 (계약번호, 설치주소)
csv_path = "/content/drive/MyDrive/시설/유지/4월마감 조건추출 선택컬럼_2503.csv"
df_csv = pd.read_csv(csv_path, encoding='utf-8')  # 필요시 'cp949'
df_csv.columns = df_csv.columns.str.strip()
df_csv = df_csv.rename(columns={df_csv.columns[3]: '계약번호', df_csv.columns[12]: '설치주소'})  # D열, M열
df_csv = df_csv[['계약번호', '설치주소']]
df_map = df_csv.drop_duplicates(subset='계약번호')  # 중복 계약번호 처리

# ✅ 5. 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1QZ2sZf2aFVRcdeeXxr8vDlRnpuN35_ZSPWzM3JVwAOw/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)

# ✅ 6. 시트 업데이트 함수
def update_sheet(sheet_name):
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_values()
    header = data[0]
    rows = data[1:]

    # ✅ 중복된 헤더 처리
    header_counts = Counter(header)
    temp_counts = {}
    new_header = []
    for h in header:
        if header_counts[h] == 1:
            new_header.append(h)
        else:
            temp_counts[h] = temp_counts.get(h, 0) + 1
            new_header.append(f"{h}_{temp_counts[h]}")

    # ✅ 데이터프레임 생성
    df_sheet = pd.DataFrame(rows, columns=new_header)

    # ✅ 계약번호 컬럼 찾기
    contract_col = [col for col in df_sheet.columns if '계약번호' in col or 'U' in col]
    if not contract_col:
        raise ValueError("계약번호 컬럼이 없습니다.")
    df_sheet['계약번호'] = df_sheet[contract_col[0]].astype(str)

    # ✅ 설치주소 매핑
    df_sheet['설치주소'] = df_sheet['계약번호'].map(df_map.set_index('계약번호')['설치주소'])

    # ✅ 업로드
    worksheet.clear()
    set_with_dataframe(worksheet, df_sheet)

# ✅ 7. 실행: 단일 시트 업데이트
update_sheet('관리고객활동대상_5월,6월')

In [12]:
# ✅ 1. 필수 라이브러리 설치 (최초 1회만)
!pip install --upgrade gspread gspread_dataframe oauth2client --quiet

# ✅ 2. 라이브러리 임포트
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from collections import Counter

# ✅ 3. 인증 설정
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)

# ✅ 4. CSV 불러오기 (계약번호 D열, 설치주소 M열)
csv_path = "/content/drive/MyDrive/시설/유지/4월마감 조건추출 선택컬럼_2503.csv"
df_csv = pd.read_csv(csv_path, encoding='utf-8')  # 또는 'cp949' 필요시
df_csv.columns = df_csv.columns.str.strip()
df_csv = df_csv.rename(columns={df_csv.columns[3]: '계약번호', df_csv.columns[12]: '설치주소'})
df_csv = df_csv[['계약번호', '설치주소']]

# 계약번호 공백 제거 및 중복 제거
df_csv['계약번호'] = df_csv['계약번호'].astype(str).str.strip()
df_map = df_csv.drop_duplicates(subset='계약번호')

# ✅ 5. 스프레드시트 열기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1QZ2sZf2aFVRcdeeXxr8vDlRnpuN35_ZSPWzM3JVwAOw/edit"
spreadsheet = gc.open_by_url(spreadsheet_url)

# ✅ 6. 시트 업데이트 함수
def update_sheet(sheet_name):
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_values()
    header = data[0]
    rows = data[1:]

    # 중복 열 이름 처리
    header_counts = Counter(header)
    temp_counts = {}
    new_header = []
    for h in header:
        if header_counts[h] == 1:
            new_header.append(h)
        else:
            temp_counts[h] = temp_counts.get(h, 0) + 1
            new_header.append(f"{h}_{temp_counts[h]}")

    # 데이터프레임 생성
    df_sheet = pd.DataFrame(rows, columns=new_header)

    # 계약번호 컬럼 찾기
    contract_col = [col for col in df_sheet.columns if '계약번호' in col or 'U' in col]
    if not contract_col:
        raise ValueError("계약번호 컬럼이 없습니다.")
    df_sheet['계약번호'] = df_sheet[contract_col[0]].astype(str).str.strip()

    # 설치주소 매핑
    df_sheet['설치주소'] = df_sheet['계약번호'].map(df_map.set_index('계약번호')['설치주소'])

    # 설치주소 컬럼을 계약번호 오른쪽으로 이동
    insert_idx = df_sheet.columns.get_loc('계약번호') + 1
    col = df_sheet.pop('설치주소')
    df_sheet.insert(insert_idx, '설치주소', col)

    # 매핑 실패 확인
    unmatched = df_sheet[df_sheet['설치주소'].isna()]
    print(f"총 {len(df_sheet)}건 중 매핑 실패 {len(unmatched)}건")
    if not unmatched.empty:
        print("매핑 실패 예시:")
        print(unmatched[['계약번호']].head())

    # 시트 클리어 후 재업로드
    worksheet.clear()
    set_with_dataframe(worksheet, df_sheet)

# ✅ 7. 실행
update_sheet('관리고객활동대상_5월,6월')

총 1277건 중 매핑 실패 7건
매핑 실패 예시:
         계약번호
74   51788153
346  51038763
708  51847602
738  51933710
918  51452236


In [14]:
# ✅ 필수 패키지 설치__인허가  영업기회
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리 불러오기
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1QZ2sZf2aFVRcdeeXxr8vDlRnpuN35_ZSPWzM3JVwAOw/edit"
SHEET_NAME = "관리고객활동대상_5월,6월"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 가져오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시군구, 읍면동 분리 함수
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API를 통한 위도/경도 추출
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리 위경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 좌표, 지도링크 URL, 길찾기 하이퍼링크 추가
df["위치좌표(위도,경도)"] = df.apply(
    lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크_URL"] = df.apply(
    lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("{row["지도링크_URL"]}", "길찾기")' if row["지도링크_URL"] else "", axis=1
)

# ✅ 결과 저장 (시트 맨 오른쪽에 추가)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.


In [ ]:
1.메인 스프레드스트 : https://docs.google.com/spreadsheets/d/1MbKqaxETuXB70C0Ucni3a40la4ymdSBlOifSdx-rYQE/edit?usp=sharing"


시트명 : 2025년0512_전사 정지조건리스트_완성_중복제거
2. 서브 스프레드시트 : https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit?usp=sharing
시트명 : DB_0414

요청사항 :  1. 메인 스프레드시트 ak ~ an 열 위도, 경도, 위치좌표(위도,경도), 지도링크_URL 컬럼에 비어 있는것을
2. 서브 스프레드시트 AP~AS 열 위도, 경도, 위치좌표(위도,경도), 지도링크 것을 1. 메인 스프레드시트 맨 우측에 적용